In [1]:
import obj
import women_sim
import pandas as pd
import numpy as np
import os
import warnings
from scipy.stats import norm
import itertools
warnings.filterwarnings('ignore')
import concurrent.futures
import time

sims = pd.read_csv('w_sims.csv')

In [2]:
# Getting all Combos of USA Women --> combos

us_women = obj.w_country('USA').ath_perf
low = {}
top_5_AA_US = dict(sorted(us_women.items(), key=lambda item: sum(item[1]['mean']), reverse = True)[:5])
for ath in top_5_AA_US:
    for app in top_5_AA_US[ath].itertuples():
        if app.Index not in low:
            low[app.Index] = [app.mean]
        else:
            low[app.Index].append(app.mean)
low_4 = {k: sorted(v)[2] for k, v in low.items()}

considered_us_women = set()
for ath in us_women:
    for app in us_women[ath].itertuples():
        if app.mean >= low_4[app.Index]:
            considered_us_women.add(ath)
combos = list(itertools.combinations(list(considered_us_women), 5)) # len() =  CHOOSE 5
# women are considered for lineup combinations if, in ANY apparatus, their average score is better than or equal to 3 of the top 5 All Around US women

In [3]:
# function to simulate a qualifying round for a given combo of USA men

USA_women_performances = obj.w_country('USA').ath_perf
def set_lineup_from_names(name_combo):
    country_comp = {'FX': [], 'BB': [], 'UB': [], 'VT': []}
    country_mean_scores = {'FX': [], 'BB': [], 'UB': [], 'VT': []}
    score_range = np.arange(12, 17, 0.001)
    for gymnast in name_combo:
        for apparatus in USA_women_performances[gymnast].itertuples():
            if len(country_comp[apparatus.Index]) < 4:
                dist = norm.cdf(score_range, loc=apparatus.mean, scale=apparatus.std_dev)
                country_comp[apparatus.Index].append((apparatus.mean, apparatus.std_dev, dist, gymnast, 'USA'))
                country_mean_scores[apparatus.Index].append(apparatus.mean)

            elif apparatus.mean > min(country_mean_scores[apparatus.Index]):
                dist = norm.cdf(score_range, loc=apparatus.mean, scale=apparatus.std_dev)
                country_comp[apparatus.Index].append((apparatus.mean, apparatus.std_dev, dist, gymnast, 'USA'))
                for i, p in enumerate(country_comp[apparatus.Index]):
                    if min(country_mean_scores[apparatus.Index]) == p[0]:
                        country_comp[apparatus.Index].pop(i)       
    qual_scores = {}
    for apparatus in country_comp: # Run Qualifying Rounds
        for athlete in country_comp[apparatus]:
            qual_scores[athlete[3] + '-' + athlete[4] + '_' + apparatus] = np.random.normal(athlete[0], athlete[1], 100)
    return pd.DataFrame(qual_scores)

In [4]:
t = time.time()
sim_results = []
def simulate_simulation(simulation_id):
    if simulation_id % 10 in list(range(os.cpu_count())):
        print(simulation_id, time.time() - t)
        pd.DataFrame(sim_results).to_csv(f'results_{simulation_id%10}.csv')
    us_sims = set_lineup_from_names(combos[simulation_id])
    full_competition = pd.concat([sims, us_sims], axis = 1)
    final_medals = women_sim.qual_to_medals(full_competition)
    medals = {'gymnasts': combos[simulation_id]}
    for comp_type in final_medals:
        for medal_type in final_medals[comp_type]['USA']:
            medals[comp_type + ' ' + medal_type] = final_medals[comp_type]['USA'][medal_type]
    sim_results.append(medals)
    return medals

# Number of simulations
num_simulations = len(combos)

# Number of parallel processes (cores)
num_cores = os.cpu_count()

# Create a list of simulation IDs
simulation_ids = list(range(num_simulations))

# Function to run simulations concurrently
def run_simulations_parallel(simulation_ids):
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_cores) as executor:
        # Using executor.map to parallelize the simulations
        results = list(executor.map(simulate_simulation, simulation_ids))

    return results

# Run simulations concurrently
simulation_results = run_simulations_parallel(simulation_ids)

# Print the results (replace with actual processing of results)
print("Simulation Results:", simulation_results)
print(t - time.time())

0 0.052973031997680664
2 3 0.05845499038696289
0.0602059364318847661 
0.058087825775146484
10 208.6139430999756
11 208.71240305900574
12 312.8997769355774
13 312.9770357608795
20 521.8007814884186
21 522.3140895366669
22 522.5440261363983
23 523.0639185905457
30 732.6919982433319
31 732.7216126918793
32 835.3720774650574
33 837.1143937110901
40 1043.3010954856873
41 1045.6079270839691
42 1046.5276882648468
43 1046.8142523765564
50 1255.812825679779
51 1256.1710977554321
52 1356.9529345035553
53 1359.4476466178894
60 1565.7055909633636
61 1568.5788462162018
62 1569.7381393909454
63 1569.8824508190155
70 1778.5949811935425
71 1779.78249168396
72 1879.4951133728027
73 1881.776701927185
80 2088.2287242412567
81 2090.5092182159424
82 2092.224947452545
83 2093.553364276886
90 2301.144958972931
91 2303.033017873764
92 2401.4574048519135
93 2404.3043682575226
100 2610.7758843898773
101 2613.024592399597
102 2614.2062180042267
103 2616.8009464740753
110 2823.09330368042
111 2826.5161838531494
1

In [5]:
pd.DataFrame(simulation_results).to_csv('women_medal.csv')